In [1]:
from astropy.io import fits
from numpy.linalg import inv as inv
import miricoord.imager.mirim_tools as mt

In [2]:
v2shift = +0.0302
v3shift = +0.1426

In [3]:
# Old distortion file
mt.set_toolversion('flt3')
oldfile=mt.get_fitsreffile()
print(oldfile)

/Users/dlaw/jwcode/miricoord/data/fits/flt3/MIRI_FM_MIRIMAGE_DISTORTION_SS.06.00.fits


In [4]:
# New Imager distortion file
new_imafile=oldfile.replace('flt3/MIRI_FM_MIRIMAGE_DISTORTION_SS.06.00.fits','flt9/MIRI_FM_MIRIMAGE_DISTORTION_SS.09.00.fits')

In [5]:
# Read in the old reference file and update the boresight reference point in the TI matrix
hdu=fits.open(oldfile)
tidata=hdu['TI matrix'].data
tidata[0,2] += v2shift
tidata[1,2] += v3shift

# Update the T matrix for the reverse transform
tdata=inv(tidata)

# Put the updated matrices in the file
hdu['TI matrix'].data = tidata
hdu['T matrix'].data = tdata
hdu[0].header['HISTORY']='20250606: Shift V2/V3 boresight to match latest FGS calibration.'
hdu.writeto(new_imafile,overwrite=True)

In [6]:
# Test where a reference pixel used to be
mt.set_toolversion('flt3')
v2old,v3old = mt.xytov2v3(692.5,511.5,'F770W')
v2old,v3old

(array([-453.37849012]), array([-373.8105493]))

In [7]:
# Test where a reference pixel moved to
mt.set_toolversion('flt9')
v2new,v3new = mt.xytov2v3(692.5,511.5,'F770W')
v2new,v3new

(array([-453.34829012]), array([-373.6679493]))